# Lab 5 - Localization
## Introduction
(*Open this notebook in Chrome or Firefox to avoid incorrect equation rendering in Safari*)

**This lab has 3 due dates: Wednesday, April 1, Wednesday, April 8, and Friday, April 3**
- **April 1** - Part A
- **April 8** - Parts B, C, D; Briefing Recordings.
- **April 10** - Part E; Lab report

Determining a robot’s orientation and position in a known environment, also known as localization, is a critical problem in the field of robotics. As is common in robotics, this seemingly simple problem is surprisingly difficult, and remains an active research area. In this lab, you will solve robotic localization by implementing Monte Carlo Localization (aka MCL or particle filter). This is a challenging lab and we'd recommend starting early and moving fast. A good goal would be to get the particle filter working before break and use the time after break to tune and write up your report/prepare your briefing

This lab consists of five parts, two of which are optional. Part A and Part E a must be submitted individually:

-   **Part A** - (Writing Assignment) Understand the motion and sensor model.
-   **Part B** - (Programming Assignment) Develop and test the particle filter algorithm in simulation.
-   **Part C** - (Autograder) Upload your solution to Gradescope for evaluation.
-   **Part D** - (**OPTIONAL: Extra Credit**) From localization to SLAM: Configure and run **Google Cartographer** in the simulator.
-   **Part E** - (**OPTIONAL: Extra Credit**) Derive the Bayes' Filter presented in Lecture 10.

<img src="figures/pf.png" width="600">

## Submission 
<a id='submission'></a>
**Part A** - (**INDIVIDUAL EFFORT**) Submit your answers **INDIVIDUALLY** to the writing assignment on **gradescope**, preferably $\LaTeX$ed. You must show work (derivations, formulas used, etc.) to receive full credit. You may collaborate on problems but you cannot simply share answers. You must write up your solutions independently.

**Part B** - (**TEAMWORK**) Implement MCL in the simulator. Augment the simulated odometry data with various types of noise and compare how your solution does compared to the ground truth odometry.

Deliverables from part B:

   -   Numerical evidence that your algorithm is working in the form of charts/data
		-   Convergence rates, cross track error, etc
		-   The simulator odometry is perfectly accurate, so that can be your ground truth! Show how the robust the simulator is in the presence of noisy odometry.
   -   An [illustrative video](https://www.youtube.com/watch?v=-c_0hSjgLYw&t=6s) of your particle filter working in a simulation environment, overlaying:
		-  Visualization of inferred position
		-   Visualization of the particle distribution
		-   The known map
		-   Laser scan data in the coordinate frame of your inferred position (it should align fairly well with the walls in the known map)
		-   Any other byproducts of your algorithm which you find worthy of visualization

**Part C** - (**TEAMWORK**) Run your implementation of MCL from part B on the autograder on Gradescope. Points will be assigned based on performance as compared to the TA solution. For more information, see Part C instructions below.

**Part D** (**TEAMWORK**, *OPTIONAL EXTRA-CREDIT*) Configure and run Google Cartographer in the simulation evnironment.
- Build a map of the Bldg 31 OR STATA basement using Cartographer (these are the usual provided maps)
- Localize using Cartographer
- Show visualization evidence that Cartographer is working in the simulator

**Part E** (**INDIVIDUAL EFFORT**, *OPTIONAL EXTRA-CREDIT*) Derive the form of the Bayes' Filter presented in Lecture 10. This is submitted as a typed PDF uploaded to Gradescope.

## Grading (10+1+1 points)
**Part A**. Writing Assignment (3 points)
- 1 point for part (i) in question 1 
- 1 point for part (ii) in question 1
- 1 point for question 2 

The 3 points of part A will be assigned based on the submitted written exercises. 

**Part B**. Programming implementation (4 points)
- 1 point for motion model 
- 1 point for sensor model 
- 1 point for MCL algorithm 
- 1 point for integration, visualization and overal performance 

Points will be assigned also depending on the presentation in the report and briefing.

**Part C**. Autograder (3 points) See Part C below for more information.

**Part D** (**Bonus**). Experiment with SLAM via running Cartographer in your simulation environment. Include your results in your lab report and presentation (1 point)

**Part E** (**Bonus**). Derive the form of the Bayes' Filter presented in Lecture 10. Submit to Gradescope. (1 point)

The grade for Parts A, B, and C are combined and averaged with the report and presentation grades. Parts B and C are weighted at 0.4, the report is weighted at 0.4, and the presentation is weighted at 0.2.

Rubric for the presentation:
- Numerical evidence that your algorithm is working in the form of charts/data (6 points)
    - Convergence rates, cross track error, etc (3 points)
    - Show how the robust the simulator is in the presence of noisy odometry (3 points)

- An illustrative video of your particle filter working in a simulation environment, overlaying (4 points)
    - Visualization of inferred position (1 points)
    - Visualization of the particle distribution (1 points)
    - The known map (1 points)
    - Laser scan data in the coordinate frame of your inferred position (it should align fairly well with the walls in the known map) (1 points)

## Part A - Writing Assignment
Work on the following questions and submit your numeric answers along with your justifications to the Gradescope assignment **Lab 5 Part A: Individual Submission**. You may check your numeric results by putting your answers in `solutions_go_here.py` and uploading to the Gradescope autograder **Lab 5 Part A: OPTIONAL**, but your grade will be based only on your explanations and answers submitted for the written portion. Just writing the final answer will not give total credit, even if correct. These questions will help you understand the algorithm before diving into coding.

<a id='question1'></a>
**Question 1**. (**Motion Model**) Consider a deterministic motion model (no added noise) based on odometry information. The motion model, $g$, takes as arguments the old particle pose, $\mathbf{x}_{t-1}$, as well as the current odometry data, $\Delta\mathbf{x}$, and returns a new pose, $\mathbf{x}_t$ with the odometry “applied” to the old poses:

$$
\mathbf{x}_t = 
    \begin{bmatrix}
        x_t \\
        y_t \\
        \theta_t \\
    \end{bmatrix}
= g(\mathbf{x}_{t-1}, \Delta x)
$$

Note that the poses $\mathbf{x}$ are expressed in the world frame relative to some arbitrary origin. The odometry $\Delta \mathbf{x}$ is necessarily a local quantity, expressed relative to the previous frame.

**i** Suppose you are given $\mathbf{x}_{t-1} = \left[0, 0, \frac{\pi}{6}\right]^T$ and $\mathbf{x}_{t} = \left[0.2, 0.1, \frac{11\pi}{60}\right]^T$. Compute the odometry data $\Delta \mathbf{x}$ (expressed in the body frame at time t-1) that results in this transformation.

**ii**. Now suppose you received the odometry data $\Delta\mathbf{x}$ from part **i**, but your car was previously at position $\mathbf{x}_{t-1} = \left[3, 4, \frac{\pi}{3}\right]^T$. Compute the current pose $\mathbf{x}_t$.

If you were to use this deterministic motion model in your particle filter all of your particles would end up in the same place - which defeats the purpose of having so many. When you build your actual motion model, you will be injecting noise into the function $g$ which will make your particles spread out as the car moves. This accounts for any uncertainty that exists in the odometry information. The sensor model will collapse this spreading of particles back into a small region.

<a id='question2'></a>
**Question 2**. (**Sensor Model**) The sensor model, $p\left(z_{t}| x_{t}, m\right)$, defines how likely it is to record a given sensor reading $z_{t}$ from a hypothesis position $x_{t}$ in a known, static map $m$ at time $t$. We will use this likelihood to "prune" the particles as the motion model tries to spread them out. Particles will be kept if the sensor reading is consistent with the map from the point of view of that particle. The definition of this likelihood is strongly dependent on the type of sensor used - a laser scanner in our case.

In a lidar model, typically, there are a few cases to be modeled in determining $p\left(z_{t}| x_{t}, m\right)$:

1. Probability of detecting a known obstacle in the map
2. Probability of a short measurement. Maybe due to internal lidar reflections (scratches or oils on the surface), hitting parts of the vehicle itself, or other unknown obstacles (people, cats, etc).
3. Probability of a very large (aka missed) measurement. Usually due to lidar beams that hit an object with strange reflective properties and did not bounce back to the sensor
4. Probability of a completely random measurement. Just in case of an asteroid strike.
    
We typically represent (1) with a gaussian distribution centered around the ground truth distance between the hypothesis pose and the nearest map obstacle. Thus, if the measured range exactly matches the expected range, the probability is maximum. If the measured range is $z_{t}$ and the ground truth range is determined (via ray casting on the map $m$ from pose $x_t$) to be $z_{t}^*$ then we have that:

$$
	p_{hit}(z_{t}| x_{t}, m)  = \begin{cases}
	\eta \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(z_t - z_t^*)^2}{2\sigma^2}\right)  &   \text{if} \quad 0 \leq z_{t} \leq z_{max}\\
    0   &   \text{otherwise}
    \end{cases}
$$
where $\eta$ is a normalization constant need to be determined.

Case (2) is represented as a downward sloping line as the ray gets further from the robot.
This is because if the unknown obstacles (people cats, etc.) are distributed uniformly in the environment, the lidar is more likely to hit ones that are closer (think about how area scales...). This likelihood can be modeled as:

$$
	p_{short}\left(z_{t}| x_{t}, m\right) =  \frac{2}{z_t^*} \begin{cases}
         1 - \frac{z_{t}}{z_{t}^*}   &   \text{if} \quad 0 \leq z_{t} \leq z_{t}^*\\
         0   &   \text{otherwise}
\end{cases}
$$

Case (3) is represented by a large spike in probability at the maximal range value, so that reflected measurements do not significantly discount particle weights.

$$
	p_{max}(z_{t}| x_{t}, m)  =\begin{cases}
	1  &  \text{if} \quad z_t = z_{max}\\
	0  &  \text{otherwise} 
	\end{cases}
	$$

Case (4) is represented by a small uniform value, to account for unforeseen effects.

$$
	p_{rand}(z_{t}| x_{t}, m)  = \begin{cases}
	\frac{1}{z_{max}}  &  \text{if} \quad 0\leq z_{t} < z_{max}\\
	0                            & \text{otherwise} 
	\end{cases}
$$
	
These four different distributions are now mixed by a weighted average, defined by the parameters $\alpha_{hit}, \alpha_{short},\alpha_{max},\alpha_{rand}$:

$$
	 p(z_{t}| x_{t}, m)  = \alpha_{hit} \cdot p_{hit}(z_{t}| x_{t}, m)  + \alpha_{short} \cdot p_{short}(z_{t}| x_{t}, m)  + \alpha_{max} \cdot p_{max}(z_{t}| x_{t}, m)  + \alpha_{rand} \cdot p_{rand}(z_{t}| x_{t}, m) 
$$

Note that in order for $p(z_{t}| x_{t}, m)$ to be a probability distrubution we have have that:

$$
\alpha_{hit}+\alpha_{short}+\alpha_{max}+\alpha_{rand}=1
$$

All together this should look like this:

<img src="figures/sensor_model_slice2.png" width="600">
	 
Find the values of $p(z_{t} | x_{t}, m)$ for the values of $z_t$ below and the following parameters:
$\alpha_{hit} = 0.74$,
$\alpha_{short}=0.07$,
$\alpha_{max}=0.07$,
$\alpha_{rand}=0.12$,
$\sigma=0.5\text{ m}$
$z_{max}=10\text{ m}$, and
$z_{t}^{*} = 7\text{ m}$.

**i**.    $z_{t} = 0\text{ m}$

**ii**.   $z_{t} = 3\text{ m}$

**iii**.  $z_{t} = 5\text{ m}$

**iv**.   $z_{t} = 8\text{ m}$

**v**.    $z_{t} = 10\text{ m}$

(*Strictly speaking, $p_{max}$ is a probability mass function, we cannot add it directly to the probability density functions $p_{hit}, p_{short}$ and $p_{rand}$. However, $p_{max}$ only affects the results at $z_{t} = z_{max} = 10\text{ m}$. For this question, you have two options*:

*a. You can use the generalized pdf to derive the expression, plug in values, and only check your answers on the autograder when $z_t = 0, 3, 5, 8 \text{ m}$. Please ignore the value at $z_{t} = 10 \text{ m}$ since the Dirac delta function is defined by its integral and does not have a function value at $z_{t} = z_{max} = 10\text{ m}$ in the traditional sense.*

*b. You can also add $p_{max}$ directly to $p(z_{t} | x_{t}, m)$. Just keep in mind that $p_{max}$ is zero except at $z_t = z_{max}$ when you calculate the probability $P(z_{t} | x_{t}, m)$ (not pdf $p(z_{t} | x_{t}, m)$) in part B*. For example, when you discretize the continuous probability density function and calculate the probability of $P_{hit} = p_{hit} \cdot \Delta z$, $P_{short} = p_{short} \cdot \Delta z$, $P_{rand} = p_{rand} \cdot \Delta z$. For discrete variable, the probability of $P_{max} = p_{max}$.

*As our ultimate goal is to evaluate the probability instead of the pdf, both options should end up with the same precomputed table in part B.*
)

## Part B - Programming Assignment

### 1. Getting Started

Grab the skeleton code from this repository and stick it in your ```src``` folder.

#### 1.1 Installing Dependencies

We are helping you out a little bit by taking care of ray tracing for you. This step needs to be repeated many many times and it can be quite slow without the help of C++. Plus it happens to be the same exact code used to generate the simulated scan in the racecar simulator!

First, make sure you have an up-to-date version of the racecar simulator installed by pulling the latest changes from GitHub (the VM image we provided may have contained an outdated version).
Then, run ```catkin_make``` **and** ```catkin_make install```

    cd ~/racecar_ws/src/racecar_simulator
    git pull
    cd ~/racecar_ws
    catkin_make
    catkin_make install
    source devel/setup.bash
    
Now you just need to build the code for the lab. We're doing this in place:

    sudo apt-get install cython
    cd ~/racecar_ws/src/localization/src/localiation/
    python setup.py build_ext --inplace

### 2. Implementation

#### Unit Testing

This lab comes with a unit-test suite that is designed to help your implementation stay on the right track. The skeleton code comes with several method definitions, and the API for these must be respected in order to pass the unit tests.

The unit tests can be found in the `/test` directory; `test_motion_model.py` and `test_sensor_model.py` contain unit tests for their respective python files in the `/src` directory which you will be modifying for your implementation.

You can run the full unit-test suite using the provided test-launch file:

```python
rostest localization unit_tests.test
```

This will start the `racecar-simulator` with a small testing map, place the robot in a predetermined location, and run the unit-test files within the ROS environment. You will be able to see the test results in the terminal used to run the launch file.

These will not be graded; they are only for your benefit. If your solution does not pass these unit tests, please reach out to the staff on piazza to discuss.

Both the motion model and the sensor model have random noise added in order to capture the probabilistic nature of the localization procedure. However, in order for us to test your implementation, we will need you to implement both of these as probabilistic functions with a deterministic flag. That is, if the `deterministic` parameter is set to true, you will not add random noise to your particle evaluations and updates. If it is false, then add random noise to particle evaluations and updates. This param can be toggled in your `/params.yaml` located in the root directory of this package.

Please **do not change the function signatures** (inputs and outputs) of the functions that we have provided in the skeleton code. If you do, the unit tests will fail. Feel free to define any other helper functions you may need.

#### 2.1 Motion Model

Implement your motion model in the ```motion_model.py``` file.

While you could use the IMU in the motion model, we recommend using the wheel odometry coming from dead-reckoning integration of motor and steering commands. It will probably be less noisy than the IMU except in extreme operating conditions. You may assume that the motor commands relate to the movement of your car according to the [geometric Ackermann model](http://www.ri.cmu.edu/pub_files/2009/2/Automatic_Steering_Methods_for_Autonomous_Automobile_Path_Tracking.pdf), with some noise due to tire slip and other imperfections.

The `/odom` topic in the simulator and the `/vesc/odom` topic on the car already give you odometry values, expressed both as an instanateous velocity and angular velocity, as well a pose in a global frame that has been accumulated over time. See the [Odometry message](http://docs.ros.org/melodic/api/nav_msgs/html/msg/Odometry.html) for more information

The precise definition of your motion model is up to you. The main constraints are:

- You must add random noise! Otherwise, as mentioned earlier, the particles won't spread out.
- As the noise approaches zero, the motion model should become exactly equal to the deterministic model you worked with in the writing section.

You might empirically determine your noise coefficients based on what works, or could try to gather data from the car which allows you to directly determine your measurement uncertainty. You might also consider how your steering commands correspond to changes in state and use this information in fusion with your odometry data for a more accurate model.

We recommend you start with the formulas you derived in [Question 1 of part A](#question1) and think of ways to add noise to these equations in a way that makes sense geometrically. The algorithm ```sample_motion_model_odometry``` in the [Probabilistic Robotics textbook](https://docs.ufpr.br/~danielsantos/ProbabilisticRobotics.pdf) may also help although it is not necessarily accurate to the car's geometry. This book is a great resource on many parts of this lab.

#### 2.2 Sensor Model

Implement your sensor model in the ```sensor_model.py``` file.

Once you have updated particle positions via your motion model, you use the sensor model to assign likelihood weights to each particle. This allows good hypotheses to have better likelihood of being sampled on the next iteration, and visa versa for bad hypotheses.

In the starter code, we've provided function signatures for `evaluate` and `precompute_sensor_model` please make sure you implement these without changing the function signature. You can check that these are working properly by running `rostest localization unit_tests.test`.


#### Precomputing the model

You may have noticed while doing the writing section that the sensor model is quite complicated and requires a lot of multiplications and non-trivial functions. And in the end this function is something that is "guessed". It will never reflect the true distribution and so long as it has the general shape that is good enough. So rather than evaluating this function millions of times at runtime, you can evaluate it on a discretized grid of inputs $z_t$ and $z_t^*$ and store these values in a lookup table.

This strategy is described in section 3.4.1 of [2]. The motivation is twofold:

1. Reading off a probability value from a table based on your actual sensor measurement and raycast ‘ground truth’ values is **MUCH** faster than doing all the math every time;
2. This provides a good opportunity to numerically normalize the $p(z_{t} |x_t, m)$ function (make sure your probabilities sum to 1).

If you plot your surface you should get something like this:

<img src="figures/sensor_model2.png" width="600">


#### Ray casting

As we mentioned, we are doing ray casting for you. If you pass an array of particles (and $n\times 3$ matrix) into our ray tracer you will get an $n\times m$ matrix where $m$ is the number of lidar beams. This is essentially a stack of $n$ lidar messages which you are probably very familiar with at this point. There is one lidar message for each particle, which is the scan that would be seen from that point.

For example if you wanted to call it on a bunch of particles that are all $(0, 0, 0)$ you could do:

    poses = np.zeros((n, 3))
    scans = self.scan_sim.scan(poses)

#### 2.3 Putting it all together: The particle filter

Once you have a motion model and a sensor model you can use them to construct the complete MCL algorithm. Do this in the ```particle_filter.py``` file.

From a very high level this will work as follows:

- Whenever you get odometry data use the motion model to update the particle positions
- Whenever you get sensor data use the sensor model to compute the particle probabilities. Then resample the particles based on these probabilities
- Anytime the particles are update (either via the motion or sensor model), determine the "average" (term used loosely) particle pose and publish that transform.

You will also consider how you want to initialize your particles. We recommend that you should be able to use some of the interactive topics in rviz to set an initialize "guess" of the robot's location with a random spread of particles around a clicked point or pose. Localization without this type of initialization (aka the global localization or the [kidnapped robot problem](https://en.wikipedia.org/wiki/Kidnapped_robot_problem)) is very hard.

As for determining the "average pose" of all of your particles, be careful with taking the average of $\theta$ values. See this page: [mean of circular quantities](https://en.wikipedia.org/wiki/Mean_of_circular_quantities). Also consider the case where your distribution is multi modal - an average could pick a very unlikely pose between two modes. What better notions of "average" could you use?

Publish this pose as a transformation between the `/map` frame and a frame for the expected car's base link. In the simulator publish to `/base_link_pf` - otherwise it will conflict with the simulator which publishes the ground truth transformation from `/map` to `/base_link`. On the car just publish to `/base_link` - this will make your lidar scan line line up with the map which can be cool to visualize!

You may find the [numpy.random.choice](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html) function useful for resampling!

#### Debugging

The best way to debug the particle filter is by isolation. First, turn off your sensor model (don't ever resample). If your particles start out in a small location, they will begin to move together and slowly spread out as the travel distance increases. Remember in the absense of noise the motion model should exactly mimic the motion of the car.

Make some test cases for the sensor model. For example scans that are identical should have a very high probability. Scans that are completely different should have a very low probability. What other useful cases could you write tests for? Visualizing is super helpful. Use the [PoseArray message](http://docs.ros.org/lunar/api/geometry_msgs/html/msg/PoseArray.html)!!!!

### 3. **Test your implementation in simulation**.

The simulator publishes ground truth position so it is a great way to test your particle filter before testing it on the real car. Augment the odometry data with various types of noise and measure how your particle filter does compared to ground truth in a number of of experiments.

Record several rosbags of random driving in the simulator. You can play these back while your particle filter is running for repeated tests. 

First try adding gaussian noise to the velocity and angular velocity given by the simulated odometry (`/odom`). Plot the error of the pose estimated by your particle filter versus the ground truth position for varying levels of noise. You can get the ground truth position by listening to the transformation between `/map` and `/base_link`.

Consider making a node that adds noise to the odometry rather than hard coding it into your particle filter. What other types of noise could you add to better replicate the noise you see on the actual car? How can you improve the performance of your particle filter in the presence of this noise?

In all of these tests make sure that your position is being published in "realtime". This means **at least 20hz**. This may seem quite fast, but when you get your car to race speeds (~10 meters per second) the poses you publish could be as much as half a meter apart which will be tough for your controller to handle. Your publish rate is upper bounded by the speed of the odometry which is 50hz on the car.

### 4. Tips and Tricks

Since the algorithm must run in realtime with a large number of particles, an efficient implementation is a requirement for success. There are a few tricks you can use, primarily:

- **ROS callbacks are not necessarily thread safe**. If your motion model and sensor model are updating the particle array independently (which they probably should) you must be careful to make sure they don't conflict! The python threading library can help.

- Downsample your laser scan: you lidar has > 1000 beams but many of them are redundant. Downsample to ~100 for good performance (you could even try lower if you want). This will make the probability distribution over your state space less "peaked" and increase the number of particles you can maintain in real time (less ray casting).

- Also to make your probability distribution less peaked you could also try to "squash" your sensor model output probability by raising it to a power of less than one (1/3 for example) to make your distribution even less peaked. If your are confused by this paragraph, look at [4,5]. 

- Don't go crazy with particles: start with ~200. You can probably get your code running with thousands of particles but it will take some well crafted code to run in real time.

- Remember that your sensor model and motion model don't need to run at the same rate! The motion model is probably much faster and over short periods of time it will accurately track the motion of the car. The sensor model can correct the dift of the motion model at a slower rate if necessary.

- Use ```rostopic hz``` to check the rate at which you are publishing the expected transformation from the map to the car's position. It should be greater than 20hz for realtime performance.

- Use numpy arrays for absolutely everything - python lists → slow
    - Use numpy functions on numpy arrays to do any computations.
    - avoid Python for loops like the plague
    - [Slice indexing is your (best) friend.](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html)
    
-   Use the smallest number of operations required to perform your arithmetic

-   Avoid unnecessary memory allocations
    -   Cache and reuse important numpy arrays by setting them to the right size during initialization of your particle filter as “self” variables
    
-   Identify your critical code paths, and keep them clean
    - Conversely, don’t worry too much about code that is called infrequently
    
-   Push code to Cython/C++ if necessary
    -   But don't worry about this too much - we already did this for the ray tracing part.
    
-   Avoid excessive function calls - function call overhead in Python → slow

-   Don’t publish visualization messages unless someone is subscribed to those topics
```
    if  self.[some_pub].get_num_connections() >  0 …
```

-   USE A PROFILER to identify good candidates for optimization

-   Make sure your TX2 is running in Max-N mode for best performance (more information [here](https://www.jetsonhacks.com/2017/03/25/nvpmodel-nvidia-jetson-tx2-development-kit/)).

- If you want an even faster (albeit more complicated to interface with) ray tracer check out [range_libc](https://github.com/kctess5/range_libc). This was written by RSS TA Corey Walsh and it is heavily optimized 

## Part C - Gradescope Autograder

### Autograder

https://www.gradescope.com/courses/87663/assignments/392418

We will be evaluating your particle filter using an autograder on Gradescope. This autograder is a bit different from the ones you have used in this class so far: it will build and run your code on Gradescope. This is to ensure that all submissions are run in an environment of comparable performance, since the accuracy of your localization package will be heavily influenced by the performance of the machine it is running on. To further mitigate variations in runtime performance, the autograder will assign scores by comparing your solution with the staff solution. Since both the staff solution and your solution are run for each test case, the grading of your submission will take several minutes (don't be surprised if it takes 10 min).

To ensure that your submission can be built and executed properly in the environment on the autograder, it is important you pay attention to the following:

 * **Submission format:** A .zip archive of your `localization` package.
 * **ROS package:** Your ROS package **must** be called `localization`, and the node implementing your particle filter **must** be called `particle_filter.py`.
 * **Particle filter node:** Pay close attention to the comments provided in `particle_filter.py` regarding parameters and topics you must use. The autograder will be configuring your node through these parameters and it will expect you to publish to certain topics.

Apart from the usual ROS packages like `rospy` and `tf2`, the following Python packages will be installed in the autograder environment:

 * `scan_simulator_2d`
 * `numpy`
 * `scipy`
 * `matplotlib`
 
Feel free to use these, but if you depend on other packages, be aware that your code will not run. Please let the staff know if there are any other packages you would like to see included.
Please also keep in mind that the autograder will be running a stock installation of ROS Melodic on Ubuntu 18.04. Any hacks or modifications you may have performed on your personal installation of ROS will not be present in the autograder environment.

### Grading
1. You will be assigned 0.5 out of 3 points for passing some initial checks to ensure that you are subscribing and publishing to the correct topics (0.25 points), and that your particle filter is running at an acceptable update rate (0.25 points). Your score for the update rate test will be proportional to the rate at which you publish pose updates, with a score of 100% corresponding to an update rate matching the staff solution's update rate.
2. The remaining 2.5 points will be assigned based on your localization performance by comparing the sequence of pose estimates your particle filter produces with the true trajectory of the robot. For each test case, the mean absolute deviation from the true trajectory, $ d = \frac{1}{T} \int{| x_{est}(t) - x_{true}(t) | dt} $ will be computed, where $x_{est}(t)$ represents the estimates produced by your particle filter (interpolated using zero order hold) and $x_{true}(t)$ is the true trajectory. For each test case, this value will be computed for both your submission and the staff solution. The staff solution's value $d_{ref}$ will be used as a reference value to score your solution by, such that your final score will be $max(0, 1 - max(0, d_{submission} - d_{ref}))$. There will be three test cases, each worth 0.833 point, and every test case will include increasing amounts of noise in the odometry data.

## Part D - (OPTIONAL: Extra Credit) Simultaneous Localization and Mapping - Configure and Run Google Cartographer
Configure and run **Google Cartographer** in the simulator. [Cartographer](https://github.com/googlecartographer/cartographer) is a system that provides real-time simultaneous localization and mapping ([SLAM](https://en.wikipedia.org/wiki/Simultaneous_localization_and_mapping)) in 2D and 3D across multiple platforms and sensor configurations.  Please go to [this repository](https://github.com/mit-rss/cartographer_config), follow the instruction to configure and run cartographer in simulation.

Your goal is to generate a map that closely resembles the ground-truth map we have provided.

## Part E - (OPTIONAL: Extra Credit) Bayes Filter
The Bayes Filter presented in class in Lecture 10 consists of 2 parts:
1. The prediction step: $$\mathbb{P}(x_k | u_{1:k}, z_{1:k-1}) = \int \underbrace{\mathbb{P}(x_k | x_{k-1}, u_k)}_{\text{Motion Model}} \mathbb{P}(x_{k-1} | u_{1:k-1}, z_{1:k-1}) dx_{k-1}$$

2. The update step: $$ \mathbb{P}(x_k | u_{1:k}, z_{1:k}) = \alpha \underbrace{\mathbb{P}(z_k|x_k)}_{\text{Measurement Model}}\mathbb{P}(x_k | u_{1:k}, z_{1:k-1})$$

In the above expressions, $x_k, u_k$, and  $z_k$ represent the belief state, control input (odometry) and the sensor measurement at time $k$, respectively.

**Given** the previous belief state $x_{k-1}$, the motion model $\mathbb{P}(x_k | x_{k-1}, u_k)$, and the measurement model $\mathbb{P}(z_k|x_k)$, **derive the prediction and update formulas stated above**.
To do this, you can rely on 3 basic probability principles:
1. Bayes Rule: $$\mathbb{P}(A|B) = \frac{\mathbb{P}(A \cap B)}{\mathbb{P}(B)}$$ which states that the probability of Event A given Event B is equal to the probability of Event A and Event B over the probability of Event B.

2. The Law of Total Probability: $$\mathbb{P}(A) = \sum_{n} \mathbb{P}(A | B_{n}) \mathbb{P}(B_n)$$ which states that the probability of Event A is sum over the possible outcomes of Event B of the probability of Event A given a specific outcome of event B times the probability of that outcome.

3. The Markov Property: $$\mathbb{P}(x_{t} | x_{t-1}, u_{t}, z_{t-1}, ... u_1, z_0) = \mathbb{P}(x_t | x_{t-1}, u_{t}) $$ which states that the current belief state is only dependent on the previous belief state and previous action.

Note: We define $u_k$ to be the action that moves us from state $x_{k-1}$ to $x_k$, so the sequence of inputs starts at $u_1$.

## References

1. <a name="ThrunRobust"></a>[S. Thrun, D. Fox, W. Burgard and F. Dellaert. “Robust Monte Carlo Localization for Mobile Robots.” Artificial Intelligence Journal. 2001](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.71.6016&rep=rep1&type=pdf).
2. <a name="FoxMarkov"></a>[D. Fox, W. Burgard, and S. Thrun. “Markov localization for mobile robots in dynamic environments,” Journal of Artificial Intelligence Research, vol. 11, pp. 391427, 1999](http://www.jair.org/media/616/live-616-1819-jair.pdf).
3. <a name="FoxAdvances"></a>[D. Fox. “KLD-sampling: Adaptive particle filters,” Advances in Neural Information Processing Systems 14 (NIPS), Cambridge, MA, 2002. MIT Press](https://papers.nips.cc/paper/1998-kld-sampling-adaptive-particle-filters.pdf).
4. <a name="BagnellPracticle"></a> [D. Bagnell “Particle Filters: The Good, The Bad, The Ugly”
](http://www.cs.cmu.edu/~16831-f12/notes/F14/16831_lecture05_gseyfarth_zbatts.pdf)
5. <a name="BootsImportance"></a>[B. Boots “Importance Sampling, Particle Filters”](https://web.archive.org/web/20170209092754/http://www.cc.gatech.edu/~bboots3/STR-Spring2017/Lectures/Lecture4/Lecture4_notes.pdf)
6. <a name="WalshCDDT"></a>[C. Walsh and S. Karaman “CDDT: Fast Approximate 2D Ray Casting for Accelerated Localization”](https://arxiv.org/abs/1705.01167)